# 🔬 CR-39 Proton Irradiation — GEANT4 Simulation
**2.5 MeV protons | Detector la 3 cm | CR-39 Nuclear Track Detector**

Rulează celulele în ordine. Pasul 1 durează ~15-20 min.

## Pasul 1 — Instalare Miniforge + GEANT4

In [ ]:
%%bash
wget -q https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
bash Miniforge3-Linux-x86_64.sh -b -p /opt/mambaforge
export PATH=/opt/mambaforge/bin:$PATH
mamba --version
echo 'Mamba ready!'

In [ ]:
%%bash
export PATH=/opt/mambaforge/bin:$PATH
mamba install -y -c conda-forge geant4=11.1 cmake make gcc gxx 2>&1 | tail -10
echo 'GEANT4 installed!'

## Pasul 2 — Clone / update repo de pe GitHub

In [ ]:
REPO_URL = 'https://github.com/CostiD/CR39_Geant4.git'

import subprocess, os
if os.path.exists('/content/CR39_Geant4'):
    r = subprocess.run(['git', '-C', '/content/CR39_Geant4', 'pull'],
                       capture_output=True, text=True)
    print('git pull:', r.stdout or r.stderr)
else:
    r = subprocess.run(['git', 'clone', REPO_URL, '/content/CR39_Geant4'],
                       capture_output=True, text=True)
    print('git clone:', r.stdout or r.stderr)

## Pasul 3 — Compilare

In [ ]:
%%bash
export PATH=/opt/mambaforge/bin:$PATH
conda run -p /opt/mambaforge --no-capture-output bash -c "
  mkdir -p /content/build && cd /content/build
  cmake /content/CR39_Geant4 \
    -DCMAKE_PREFIX_PATH=/opt/mambaforge \
    -DCMAKE_BUILD_TYPE=Release 2>&1 | tail -5
  make -j2 2>&1
  ls -lh CR39_sim && echo 'Build SUCCESS!'
"

## Pasul 4 — Rulare simulare (50 000 protoni)

In [ ]:
%%bash
export PATH=/opt/mambaforge/bin:$PATH
conda run -p /opt/mambaforge --no-capture-output bash -c "
  cd /content/build
  cp /content/CR39_Geant4/run.mac .
  time ./CR39_sim run.mac
"

## Pasul 5 — Combină CSV-urile (GEANT4 MT scrie câte unul per thread)

In [ ]:
import subprocess

result = subprocess.run(
    ['find', '/content/build', '-name', 'cr39_output*.csv'],
    capture_output=True, text=True
)
files = sorted([f for f in result.stdout.strip().split('\n') if f])
print(f'Fișiere găsite ({len(files)}):', files)

out_path = '/content/cr39_output.csv'
with open(out_path, 'w') as outfile:
    header_written = False
    for f in files:
        with open(f) as infile:
            lines = infile.readlines()
        if not header_written:
            outfile.writelines(lines)
            header_written = True
        else:
            outfile.writelines([l for l in lines if not l.startswith('#')])

total = sum(1 for l in open(out_path) if not l.startswith('#'))
print(f'Total evenimente: {total:,}')

## Pasul 6 — Vizualizare publicabilă

In [ ]:
!pip install -q scipy matplotlib pandas

In [ ]:
import os
os.chdir('/content')
exec(open('/content/CR39_Geant4/analysis/visualize.py').read())

## Pasul 7 — Download rezultate

In [ ]:
from google.colab import files
files.download('/content/cr39_output.csv')
files.download('/content/cr39_proton_analysis.pdf')
files.download('/content/cr39_proton_analysis.png')